In [1]:
import os
import time
import logging
import yaml
import ast
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

from pprgo import utils
from pprgo import ppr
from pprgo import pprgo

In [2]:
# Set up logging
logger = logging.getLogger()
logger.handlers = []
ch = logging.StreamHandler()
formatter = logging.Formatter(
        fmt='%(asctime)s (%(levelname)s): %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S')
ch.setFormatter(formatter)
logger.addHandler(ch)
logger.setLevel('INFO')

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [3]:
# !wget --show-progress -O data/reddit.npz https://ndownloader.figshare.com/files/23742119

# Download dataset

# Load config

In [4]:
with open('config_demo.yaml', 'r') as c:
    config = yaml.safe_load(c)

In [5]:
# For strings that yaml doesn't parse (e.g. None)
for key, val in config.items():
    if type(val) is str:
        try:
            config[key] = ast.literal_eval(val)
        except (ValueError, SyntaxError):
            pass

In [6]:
data_file           = config['data_file']           # Path to the .npz data file
# data_file = 'data/cora_full.npz'
data_file = 'data/reddit.npz'

split_seed          = config['split_seed']          # Seed for splitting the dataset into train/val/test
ntrain_div_classes  = config['ntrain_div_classes']  # Number of training nodes divided by number of classes
attr_normalization  = config['attr_normalization']  # Attribute normalization. Not used in the paper

alpha               = config['alpha']               # PPR teleport probability
alpha = 0.5
eps                 = config['eps']                 # Stopping threshold for ACL's ApproximatePR
topk                = config['topk']                # Number of PPR neighbors for each node
ppr_normalization   = config['ppr_normalization']   # Adjacency matrix normalization for weighting neighbors

hidden_size         = config['hidden_size']         # Size of the MLP's hidden layer
nlayers             = config['nlayers']             # Number of MLP layers
weight_decay        = config['weight_decay']        # Weight decay used for training the MLP
dropout             = config['dropout']             # Dropout used for training

lr                  = config['lr']                  # Learning rate
max_epochs          = config['max_epochs']          # Maximum number of epochs (exact number if no early stopping)
batch_size          = config['batch_size']          # Batch size for training
batch_mult_val      = config['batch_mult_val']      # Multiplier for validation batch size

eval_step           = config['eval_step']           # Accuracy is evaluated after every this number of steps
run_val             = config['run_val']             # Evaluate accuracy on validation set during training

early_stop          = config['early_stop']          # Use early stopping
patience            = config['patience']            # Patience for early stopping

nprop_inference     = config['nprop_inference']     # Number of propagation steps during inference
inf_fraction        = config['inf_fraction']        # Fraction of nodes for which local predictions are computed during inference

# Load the data

In [7]:
start = time.time()
(adj_matrix, attr_matrix, labels,
 train_idx, val_idx, test_idx) = utils.get_data(
        f"{data_file}",
        seed=split_seed,
        ntrain_div_classes=ntrain_div_classes,
        normalize_attr=attr_normalization
)
try:
    d = attr_matrix.n_columns
except AttributeError:
    d = attr_matrix.shape[1]
nc = labels.max() + 1
print(nc)

# print('total: ', adj_matrix.shape[0])
print('Training: ', len(train_idx))
print('Validation: ', len(val_idx))
print('Testing: ', len(test_idx))

print('train_idx: ', train_idx)

time_loading = time.time() - start
print(f"Runtime: {time_loading:.2f}s")

70
Training:  1400
Validation:  14000
Testing:  3400
train_idx:  [   12    16    18 ... 18790 18793 18798]
Runtime: 0.31s


# Preprocessing: Calculate PPR scores

In [8]:
# compute the ppr vectors for train/val nodes using ACL's ApproximatePR
start = time.time()

topk_train, mean_kn = ppr.topk_ppr_matrix(adj_matrix, alpha, eps, train_idx, topk,
                                 normalization=ppr_normalization, S=1)
if run_val:
    topk_val = ppr.topk_ppr_matrix(adj_matrix, alpha, eps, val_idx, topk,
                                   normalization=ppr_normalization)
else:
    topk_val = None

print(topk_train.shape)
time_preprocessing = time.time() - start
print(f"Runtime: {time_preprocessing:.2f}s")

print(mean_kn)


[0.29002368450164795, 0.010774497874081135, 0.0011891918256878853, 0.0013862489722669125, 0.00029153720242902637, 0.008447730913758278, 0.0013661985285580158, 0.0012648390838876367, 0.00037985778180882335, 0.000360208738129586, 0.0003990377008449286, 0.0002971408248413354, 0.0011058419477194548, 0.0018266900442540646, 0.0005649934173561633, 0.0021793153136968613, 0.0007611322216689587, 0.0008056584047153592, 0.008751115761697292, 0.0008440665551461279, 0.0009883882012218237, 0.00037064554635435343, 0.0018193927826359868, 0.0004742582095786929, 0.0038575741928070784, 0.00022255234944168478, 0.00022255234944168478, 0.007370032835751772, 0.0009910704102367163, 0.002871639793738723, 0.0009212541044689715, 0.002500839764252305, 0.0032196417450904846, 0.0008529454353265464, 0.008003913797438145, 0.002645265543833375, 0.0024916199035942554, 0.004840475507080555, 0.007564976345747709, 0.0011696267174556851, 0.0012569293612614274, 0.00015126485959626734, 0.0004521876689977944, 0.009622791782021

# Training: Set up model and train

In [9]:
start = time.time()
tf.reset_default_graph()
tf.set_random_seed(0)

model = pprgo.PPRGo(d, nc, hidden_size, nlayers, lr, weight_decay, dropout,
                    sparse_features=type(attr_matrix) is not np.ndarray)

sess = tf.compat.v1.Session()
with sess.as_default():
    tf.compat.v1.global_variables_initializer().run()
    nepochs, loss_hist, acc_hist, f1_hist = pprgo.train(
            sess=sess, model=model, attr_matrix=attr_matrix,
            train_idx=train_idx, val_idx=val_idx,
            topk_train=topk_train, topk_val=topk_val,
            labels=labels,
            max_epochs=max_epochs, batch_size=batch_size, batch_mult_val=batch_mult_val,
            eval_step=eval_step, early_stop=early_stop, patience=patience)
time_training = time.time() - start
logging.info('Training done.')
print(f"Runtime: {time_training:.2f}s")

2022-06-15 15:09:12 (WARNING): From /var/folders/x5/bhlqr68144x_138bllx2mdwh0000gn/T/ipykernel_4946/2975614973.py:2: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.



2022-06-15 15:09:12 (WARNING): From /var/folders/x5/bhlqr68144x_138bllx2mdwh0000gn/T/ipykernel_4946/2975614973.py:3: The name tf.set_random_seed is deprecated. Please use tf.compat.v1.set_random_seed instead.

2022-06-15 15:09:14 (INFO): Epoch 6, step 20: train 3.95836
2022-06-15 15:09:14 (INFO): Epoch 13, step 40: train 3.39601
2022-06-15 15:09:15 (INFO): Epoch 19, step 60: train 3.01421
2022-06-15 15:09:16 (INFO): Epoch 26, step 80: train 2.65718
2022-06-15 15:09:17 (INFO): Epoch 33, step 100: train 2.07830
2022-06-15 15:09:18 (INFO): Epoch 39, step 120: train 1.76127
2022-06-15 15:09:19 (INFO): Epoch 46, step 140: train 1.67763
2022-06-15 15:09:20 (INFO): Epoch 53, step 160: train 1.32753
2022-06-15 15:09:21 (INFO): Epoch 59, step 180: train 1.18835
2022-06-15 15:09:22 (INFO): Epoch 66, step 200: train 1.25268
2022-06-15 15:09:23 (INFO): Epoch 73, step 220: train 1.04112
2022-06-15 15:09:24 (INFO): Epoch 79, step 240: train 0.96792
2022-06-15 15:09:25 (INFO): Epoch 86, step 260: tra

KeyboardInterrupt: 

# Inference (val and test)

In [ ]:
start = time.time()
predictions, time_logits, time_propagation = model.predict(
        sess=sess, adj_matrix=adj_matrix, attr_matrix=attr_matrix, alpha=alpha,
        nprop=nprop_inference, inf_fraction=inf_fraction,
        ppr_normalization=ppr_normalization)
time_inference = time.time() - start
print(f"Runtime: {time_inference:.2f}s")

Runtime: 23.51s


# Collect and print results

In [ ]:

acc_train = 100 * accuracy_score(labels[train_idx], predictions[train_idx])
acc_val = 100 * accuracy_score(labels[val_idx], predictions[val_idx])
acc_test = 100 * accuracy_score(labels[test_idx], predictions[test_idx])

f1_train = f1_score(labels[train_idx], predictions[train_idx], average='macro')
f1_val = f1_score(labels[val_idx], predictions[val_idx], average='macro')
f1_test = f1_score(labels[test_idx], predictions[test_idx], average='macro')



#gpu_max_bytes = tf.contrib.memory_stats.MaxBytesInUse()
#gpu_memory = sess.run(gpu_max_bytes)
memory = utils.get_max_memory_bytes()

time_total = time_preprocessing + time_training + time_inference

In [ ]:
print(f'''
Accuracy: Train: {acc_train:.1f}%, val: {acc_val:.1f}%, test: {acc_test:.1f}%
F1 score: Train: {f1_train:.3f}, val: {f1_val:.3f}, test: {f1_test:.3f}

Runtime: Preprocessing: {time_preprocessing:.2f}s, training: {time_training:.2f}s, inference: {time_inference:.2f}s -> total: {time_total:.2f}s
Memory: Main: {(memory/1024) / 2**30:.2f}GB
''')

#Memory: Main: {memory / 2**30:.2f}GB, GPU: {gpu_memory / 2**30:.3f}GB


Accuracy: Train: 36.3%, val: 27.4%, test: 27.6%
F1 score: Train: 0.174, val: 0.116, test: 0.115

Runtime: Preprocessing: 11.07s, training: 3.86s, inference: 23.51s -> total: 38.44s
Memory: Main: 3.49GB

